In [1]:
## Import base packages we'll use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from seaborn import set_style
set_style("whitegrid")

In [2]:
## this imports BeautifulSoup from its package, bs4
import bs4
## First we import the BeautifulSoup object
from bs4 import BeautifulSoup

In [207]:
df = pd.read_csv("data/kc_house_data.csv")

zips = df.zipcode.value_counts().index

zips


Index([98103, 98038, 98115, 98052, 98117, 98042, 98034, 98118, 98023, 98006,
       98133, 98059, 98058, 98155, 98074, 98033, 98027, 98125, 98056, 98053,
       98001, 98075, 98126, 98092, 98144, 98106, 98116, 98029, 98004, 98199,
       98065, 98122, 98146, 98008, 98028, 98040, 98198, 98003, 98031, 98072,
       98168, 98112, 98055, 98107, 98136, 98178, 98030, 98177, 98166, 98022,
       98105, 98045, 98002, 98077, 98011, 98019, 98108, 98119, 98005, 98007,
       98188, 98032, 98014, 98070, 98109, 98102, 98010, 98024, 98148, 98039],
      dtype='int64', name='zipcode')

In [52]:
import requests

In [68]:
response.status_code

200

In [166]:
from time import sleep

In [167]:
root = "https://www.schooldigger.com/go/WA/zip/" ##we will create our urls by adding to this root url

dfs = {} ##this will store all the dataframes

for t in range(len(zips)): ##looping through all the zipcodes and scraping respectively that webpage
    url = root+str(zips[t])+"/search.aspx"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('tbody')
    rows = table.find_all('tr')
    school_name = []

    rating = []
    
    ##glyphicon-star-empty is the only thing I found whoihc keeps track of the rating on the webpage
    ##The counts of empty subtracted from overall rating of 5 gives is the number of stars.

    for i in range(len(rows)):
        school_name.append(rows[i].find('td')['data-order'])
        rating.append(5-len(rows[i].find_all('span', attrs={'class':'glyphicon-star-empty'}))) 
    
    ##creating a dataframe and storing it in dfs so that we can call that dataframe using dfs[index]
    
    dfs[t] = pd.DataFrame({'School_Name' : school_name,
                    'Schooldigger_Rating':rating,
                    'Zipcode': (zips[t])*np.ones(len(rating))})
    
    sleep(2) ##repeated calls to the website

In [179]:
dfs[1]

,School_Name,Schooldigger_Rating,Zipcode
0,Rock Creek Elementary,5,98038.0
1,Glacier Park Elementary,4,98038.0
2,Lake Wilderness Elementary,4,98038.0
3,Shadow Lake Elementary,4,98038.0
4,Tahoma Senior High School,5,98038.0
5,Tahoma Elementary,4,98038.0
6,Cedar River Elementary,5,98038.0
7,Tahoma Open Doors,5,98038.0


In [227]:
DD1 = pd.DataFrame({}) 

In [228]:
for t in range(len(zips)):
    DD1 = pd.concat([DD1, dfs[t]], axis=0, ignore_index=True)
    

In [237]:
DD2=DD1.copy()

DD2['Zipcode'] = DD2['Zipcode'].astype(np.int64)

In [240]:
DD2.to_csv("data/kcpubschools_by_zip.csv") ## removing the .0 from the zipcodes (could have been done at the very beginning)